In [ ]:
pip install transformers sentence-transformers torch

In [ ]:
pip install datasets

In [ ]:
pip install --upgrade datasets fsspec


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("truthful_qa", "multiple_choice")

# Inspect the data
print(dataset["validation"][0])


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Convert dataset into a DataFrame
data = []
for item in dataset["validation"]:
    question = item["question"]
    for i, answer in enumerate(item["mc1_targets"]["choices"]):
        label = item["mc1_targets"]["labels"][i]  # 1 = true, 0 = hallucinated
        data.append({"question": question, "answer": answer, "label": label})

df = pd.DataFrame(data)

# Split into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

print(train_df.head())


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load a tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["answer"], padding="max_length", truncation=True)

train_encodings = tokenize_function(train_df.to_dict("list"))
val_encodings = tokenize_function(val_df.to_dict("list"))

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Compute accuracy, precision, recall, F1-score
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


# Convert to torch dataset
class HallucinationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = HallucinationDataset(train_encodings, train_df["label"].tolist())
val_dataset = HallucinationDataset(val_encodings, val_df["label"].tolist())

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


In [ ]:

from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the TruthfulQA dataset
dataset = load_dataset("truthful_qa", "multiple_choice")

# Convert the dataset into a DataFrame
data = []
for item in dataset["validation"]:
    question = item["question"]
    for i, answer in enumerate(item["mc1_targets"]["choices"]):
        label = item["mc1_targets"]["labels"][i]  # 1 = true, 0 = hallucinated
        data.append({"question": question, "answer": answer, "label": label})

df = pd.DataFrame(data)

# Split into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Load a larger model, BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["answer"], padding="max_length", truncation=True)

train_encodings = tokenize_function(train_df.to_dict("list"))
val_encodings = tokenize_function(val_df.to_dict("list"))

# Convert the tokenized data into a Dataset class
class HallucinationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = HallucinationDataset(train_encodings, train_df["label"].tolist())
val_dataset = HallucinationDataset(val_encodings, val_df["label"].tolist())

# Define a function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,  # Increased epochs to 5 for better learning
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

# Test function to classify a response
def classify_response(question, answer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(question, answer, return_tensors="pt", padding="max_length", truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    prediction = torch.argmax(outputs.logits).item()
    return "Truthful" if prediction == 1 else "Hallucinated"



In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load and prepare dataset
dataset = load_dataset("truthful_qa", "multiple_choice")
data = []
for item in dataset["validation"]:
    question = item["question"]
    for i, answer in enumerate(item["mc1_targets"]["choices"]):
        label = item["mc1_targets"]["labels"][i]
        data.append({"question": question, "answer": answer, "label": label})
df = pd.DataFrame(data)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Load tokenizer & model
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["answer"], padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_df.to_dict("list"))
val_encodings = tokenize_function(val_df.to_dict("list"))

# Dataset class
class HallucinationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = HallucinationDataset(train_encodings, train_df["label"].tolist())
val_dataset = HallucinationDataset(val_encodings, val_df["label"].tolist())

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Training config
training_args = TrainingArguments(
    output_dir="./roberta_results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()



In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

# 1) Load & prepare the TruthfulQA dataset
dataset = load_dataset("truthful_qa", "multiple_choice")
data = []
for item in dataset["validation"]:
    q = item["question"]
    for i, ans in enumerate(item["mc1_targets"]["choices"]):
        lbl = item["mc1_targets"]["labels"][i]  # 1=true, 0=hallucinated
        data.append({"question": q, "answer": ans, "label": lbl})
df = pd.DataFrame(data)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 2) Load DeBERTa‑v3
model_name = "microsoft/deberta-v3-base"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 3) Tokenize with an explicit max_length
def tokenize_function(exs):
    return tokenizer(
        exs["question"],
        exs["answer"],
        padding="max_length",      # pad everything to...
        truncation=True,           # ...and truncate longer ones
        max_length=128             # <-- set your desired length
    )

train_encodings = tokenize_function(train_df.to_dict("list"))
val_encodings   = tokenize_function(val_df.to_dict("list"))

# 4) Wrap in a torch Dataset
class HallucinationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = HallucinationDataset(train_encodings, train_df["label"].tolist())
val_dataset   = HallucinationDataset(val_encodings,   val_df["label"].tolist())

# 5) Data collator for dynamic padding (optional here since we used max_length)
data_collator = DataCollatorWithPadding(tokenizer)

# 6) Metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds  = pred.predictions.argmax(-1)
    acc    = accuracy_score(labels, preds)
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f}

# 7) Training arguments
training_args = TrainingArguments(
    output_dir="./deberta_results",
    eval_strategy="epoch",       # or evaluation_strategy="epoch" in newer versions
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# 8) Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,   # <-- ensures padding is consistent
    compute_metrics=compute_metrics,
)

# 9) Train & Evaluate
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)
